# Practice for Join Statements with SQL

Import the necessary packages and connect to the database 'data.sqlite'.

In [1]:
# Run without change

# Libraries and database

import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')

1. Select the names (frist and last) of all employees in Boston

In [10]:
# Your code here

q = """
SELECT firstName, lastName
FROM employees
JOIN offices
    USING(officeCode)
WHERE city = "Boston"
;
"""

pd.read_sql(q, conn)


,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


2. Are there any offices that have zero employees?

In [15]:
# Your code here
q = """
SELECT 
    o.officeCode, o.city,
    COUNT(e.employeeNumber) AS num_emp
FROM offices AS o
LEFT JOIN employees as e
    USING(officeCode)
GROUP BY officeCode
HAVING num_emp = 0
;
"""

pd.read_sql(q, conn)

,officeCode,city,num_emp
0,27,Boston,0


3. How many customers are there per office?

In [17]:
# Your code here
q = """
SELECT 
    o.officeCode, o.city,
    COUNT(c.customerNumber) AS num_cust
FROM offices AS o
JOIN employees as e
    USING(officeCode)
JOIN customers as c
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode
;
"""

pd.read_sql(q, conn)

,officeCode,city,num_cust
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


4. Display the names of every individual product that each employee has sold as a dataframe.

In [42]:
q1 = """
SELECT  firstName, lastName, productName
FROM orders AS o
JOIN orderDetails AS od
    USING(orderNumber)
JOIN products AS p
    USING(productCode)
JOIN customers AS c
    USING(customerNumber)
JOIN employees AS e
    ON c.salesRepEmployeeNumber = e.employeeNumber
ORDER BY employeeNumber
;
"""

df = pd.read_sql(q1, conn)

df

,firstName,lastName,productName
0,Leslie,Jennings,1937 Lincoln Berline
1,Leslie,Jennings,1936 Mercedes-Benz 500K Special Roadster
2,Leslie,Jennings,1934 Ford V8 Coupe
3,Leslie,Jennings,18th Century Vintage Horse Carriage
4,Leslie,Jennings,1917 Maxwell Touring Car
...,...,...,...
2991,Martin,Gerard,1962 Volkswagen Microbus
2992,Martin,Gerard,1936 Chrysler Airflow
2993,Martin,Gerard,1980’s GM Manhattan Express
2994,Martin,Gerard,1996 Peterbilt 379 Stake Bed with Outrigger


5. Display the number of products each employee has sold
- Alphabetize the results by employee last name.
- Use the quantityOrdered column from orderDetails.
- Think about how to group the data when some employees might have the same first or last name.


In [47]:
q = """
SELECT  firstName || " " || lastName AS full_name, SUM(quantityOrdered)
FROM orders AS o
JOIN orderDetails AS od
    USING(orderNumber)
JOIN customers AS c
    USING(customerNumber)
JOIN employees AS e
    ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY employeeNumber
;
"""

df = pd.read_sql(q, conn)

df

,full_name,SUM(quantityOrdered)
0,Leslie Jennings,11854
1,Leslie Thompson,4056
2,Julie Firrelli,4227
3,Steve Patterson,5561
4,Foon Yue Tseng,5016
5,George Vanauf,7423
6,Loui Bondur,6186
7,Gerard Hernandez,14231
8,Pamela Castillo,9290
9,Larry Bott,8205


6.  Display the names of employees who have sold more than 200 different products.

In [48]:
q = """
SELECT  firstName || " " || lastName AS full_name, COUNT(productCode) AS unique_products_sold
FROM orders AS o
JOIN orderDetails AS od
    USING(orderNumber)
JOIN customers AS c
    USING(customerNumber)
JOIN employees AS e
    ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY employeeNumber
HAVING unique_products_sold > 200
ORDER BY employeeNumber
;
"""

df = pd.read_sql(q, conn)

df

,full_name,unique_products_sold
0,Leslie Jennings,331
1,George Vanauf,211
2,Gerard Hernandez,396
3,Pamela Castillo,272
4,Larry Bott,236
5,Barry Jones,220


In [49]:
conn.close()